In [28]:
from gensim.models import Word2Vec
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vepif\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
train_data = []
test_data = []

In [30]:
def prepare_data(tokens):
    pattern = r'[^\w\s]'
    stop_words = set(stopwords.words('english'))
    
    cleaned_tokens = []
    for token in tokens:
        clean_token = re.sub(pattern, '', str(token).lower())
        if clean_token != '' and clean_token not in stop_words:
            cleaned_tokens.append(clean_token)
        
    return cleaned_tokens

In [31]:
for data_type in tqdm(['train', 'test']):
    for folder in tqdm(['0', '1', '2', '3']):
        folder_path = f'../assets/annotated-corpus/{data_type}/{folder}/{folder}.tsv'
        data = pd.read_csv(folder_path, sep='\t', header= None)
        data = prepare_data(data[0].to_list())
        
        if data_type == 'train':
            train_data.extend(data)
        else:
            test_data.extend(data)
            

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:28<00:00, 14.15s/it]


In [39]:
model = Word2Vec(sentences=train_data, vector_size=100, window=3, min_count=1, workers=-1)

In [42]:
model.train(train_data, total_examples=len(train_data), epochs=100)

(0, 0)

In [43]:
model.save("word2vec.model")

In [ ]:
model.wv['computer']

In [ ]:
model